In [1]:
import os
import sys
import torch
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import math
from tensorboardX import SummaryWriter

import import_ipynb
from CustomDataset import ControlsDataset
from Model import ConvNet

REPORT_EVERY_ITER = 20

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('using device', device)

importing Jupyter notebook from CustomDataset.ipynb
importing Jupyter notebook from Model.ipynb
units after conv 512
conv parameters:  168224
fc parameters:  164353
input torch.Size([4, 3, 480, 640])
output torch.Size([4, 1])
using device cuda


In [2]:
dataset = ControlsDataset()
dataloader = DataLoader(dataset, batch_size = 256, shuffle = True, num_workers = 0)

net = ConvNet().to(device)
print("number of parameters: ", sum(p.numel() for p in net.parameters()))

units after conv 512
conv parameters:  168224
fc parameters:  164353
number of parameters:  332577


In [3]:
epochs = 100
optimizer = optim.Adam(net.parameters(), lr=0.0001)
criterion = nn.MSELoss()
writer = SummaryWriter()
iter_no = 0

for epoch in range(epochs):
    for i_batch, sampled_batch in enumerate(dataloader):
        iter_no += 1
        images = sampled_batch['image'].to(device).float()
        controls = sampled_batch['control'].to(device).float()
    
        optimizer.zero_grad()
        prediction = net(images)
        
        loss = criterion(prediction, controls)
        loss.backward()
        optimizer.step()

        if iter_no % REPORT_EVERY_ITER == 0:
            writer.add_scalar("Loss", loss.item(), iter_no)
        
        out = "{0},{1}\tLoss:{2}\tAllocated:{3}GB\tCached:{4}GB"
        print(out.format(str(epoch),
                        str(iter_no),
                        round(loss.item(),5),
                        round(torch.cuda.memory_allocated(0)/1024**3,3),
                        round(torch.cuda.memory_allocated(0)/1024**3,3)
                        ),end='\r')

In [4]:
#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024))

GeForce GTX 1080
Memory Usage:
Allocated: 0.0 GB
Cached:    3665920
